In [ ]:
from urllib.request import urlopen
import json
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import html, Input, Output, dcc, dash_table
import random

In [ ]:
url = 'https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/russia.geojson'
with urlopen(url) as response:
    counties = json.load(response)

In [ ]:
df = pd.read_excel('mydata1.xlsx', sheet_name='Data')
topop = ['IRA_1','IRA_2','INV_1','INV_2','INV_3','LUJ_2','LUJ_4','GUL_2','DES_1']
for s in topop:
    df[s] = df[s]/df['Population']/100
df['GUL_1'] = 1/df['GUL_1']
df.describe()

In [ ]:
variables = ['AVA_1', 'AVA_2', 'AVA_3', 'IRA_1', 'IRA_2',
       'INV_1', 'INV_2', 'INV_3', 'LUJ_1', 'LUJ_2', 'LUJ_3', 'LUJ_4',
       'GUL_1', 'GUL_2', 'GUL_3', 'DES_1', 'DES_2']

In [ ]:
scaled_df = df
minimum = 0
maximum = 0
for s in variables:
    minimum = min(df[s])*0.95
    maximum = max(df[s])*1.05
    scaled_df[s] = (df[s] - minimum)/(maximum - minimum)
scaled_df.describe()

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
df_number = df.drop(labels = ['name','Population','id'],axis = 1)
d = scaler.fit_transform(df_number)
scaled2_df = pd.DataFrame(d, columns=df_number.columns.values)
scaled2_df.insert(0,'name',df['name'])
scaled2_df.insert(1,'id',df['id'])
scaled2_df.describe()

In [ ]:
sins = ['name','Жадность','Зависть','Гнев','Похоть','Чревоугодие','Уныние','Гордыня']
sins_df = pd.DataFrame(columns=sins)
sins_df['name'] = df['name']
sins_df['Жадность'] = (scaled_df['AVA_1']*scaled_df['AVA_2']*scaled_df['AVA_3'])**(1/3)
sins_df['Зависть'] = (scaled_df['INV_1']*scaled_df['INV_2']*scaled_df['INV_3'])**(1/3)
sins_df['Гнев'] = (scaled_df['IRA_1']*scaled_df['IRA_2'])**(1/2)
sins_df['Похоть'] = (scaled_df['LUJ_1']*scaled_df['LUJ_2']*scaled_df['LUJ_3']*scaled_df['LUJ_4'])**(1/4)
sins_df['Чревоугодие'] = (scaled_df['GUL_1']*scaled_df['GUL_2']*scaled_df['GUL_3'])**(1/3)
sins_df['Уныние'] = (scaled_df['DES_1']*scaled_df['DES_2'])**(1/2)
sins_df['Гордыня'] = (sins_df['Жадность']*sins_df['Зависть']*sins_df['Гнев']*sins_df['Похоть']*sins_df['Чревоугодие']*sins_df['Уныние'])**(1/6)
sins_df.sort_values(by = ['Гордыня'],ascending=True).head(10)

In [ ]:
colors = ['']*7
for i in range(6):
    colors[i] = str(input())
colors[6] = 'Plotly3'

In [ ]:
k = 0
for s in sins[1:]:
    color1 = colors[k]
    k += 1
    fig = px.choropleth_mapbox(sins_df, geojson = counties, locations = 'name',
    color= s,
    height = 600,
    width = 900,
    featureidkey='properties.name',
    mapbox_style="carto-darkmatter",
    zoom=1.5,
    center = {"lat": 70, "lon": 108},
    color_continuous_scale = color1)
    #fig.show()
    fig.write_html(f"{s}.html")
    fig.write_image(f"{s}.png")

In [ ]:
app = JupyterDash(__name__)
app.layout = html.Div(
                      [
                        html.H2("Жадность", style={'text-align': 'center'}),
                        dcc.Graph(figure=fig, id='name')
                      ]
                     )
app.run_server(mode='jupyterlab', port = 8080, dev_tools_ui=True, debug=True,
              dev_tools_hot_reload =True, threaded=True)

In [ ]:
for s in sins[1:]:
    sins_df.sort_values(by = s,ascending=True).head(5).to_excel(f"{s}_top5.xlsx")
    sins_df.sort_values(by = s,ascending=False).head(5).to_excel(f"{s}_antitop5.xlsx")